# import and set paths


In [1]:
## import functions from .py utils file: 
from Utilities.across_animals_functions import *


#Animals to include in the analysis
Animal_ID = ['EJT100','EJT...','EJT...']

# name to save this anbalysis under 
Cohort_name = 'Test_Analysis'

#Set paths
InputPath = (r'D:\Behavioural_data_output\\')

OutputPath = (r'D:\Behavioural_data_output\\')

In [2]:
# perfect score data:
AA_PerfectScores = []
AA_AttemptedSequences = []
#Transition score data:
AttemptedTs = [[],[],[],[]]
AA_TransitionScores = [[],[],[],[]]
AA_TrainingLevels= []
AA_FilteredTransitionTimes = [] 
AA_Transitionscore_trials = []
AA_TrialsPerSessions = []
AA_TransitionTimes_inin = []
AA_TransitionTimes_outin = []
AA_TrainingLevels = []

for Animal_index, CurrentAnimal in enumerate(Animal_ID):

    InputPathCurrent = (InputPath + CurrentAnimal+r'\\AcrossSessions\\AnalysisOutput\\')
    
    #load in data:
    if not os.path.isdir(InputPathCurrent):
        print('Error, do Across Sessions analsis first')
        
    # perfect score data
    with open (InputPathCurrent + 'PerfectScore_trials' , 'rb') as fp:
        PerfectCount = pickle.load(fp)        
    AA_PerfectScores = AA_PerfectScores + [PerfectCount]
    
    # Transition score data
    with open (InputPathCurrent + 'Transitionscore_trials' , 'rb') as fp:
        Transitionscore_trials = pickle.load(fp)        
    AA_Transitionscore_trials = AA_Transitionscore_trials + [Transitionscore_trials]
    
    # TrialsPerSessions data
    with open (InputPathCurrent + 'TrialsPerSessions' , 'rb') as fp:
        TrialsPerSessions = pickle.load(fp)        
    AA_TrialsPerSessions = AA_TrialsPerSessions + [TrialsPerSessions]#
    
    # Transition times in in
    with open (InputPathCurrent + 'MeanTransitionTimes_bytrial_inin' , 'rb') as fp:
        Mean_Transitions_bytrial_inin = pickle.load(fp)        
    AA_TransitionTimes_inin = AA_TransitionTimes_inin + [Mean_Transitions_bytrial_inin]
    
    # Transition times out in
    with open (InputPathCurrent + 'MeanTransitionTimes_bytrial_outin' , 'rb') as fp:
        Mean_Transitions_bytrial_outin = pickle.load(fp)        
    AA_TransitionTimes_outin = AA_TransitionTimes_outin + [Mean_Transitions_bytrial_outin]
    
    with open (InputPathCurrent + 'Training_levels' , 'rb') as fp:
        Training_levels = pickle.load(fp)        
    AA_TrainingLevels = AA_TrainingLevels + [Training_levels]

    
print('Plotting...')
    
################################################################################################################
## Perfect scores 
#plot mean + all curves Perfect scores (smoothed more)
window = 100
################################################################################################################

TrialbyTrial_Pscores = conactinate_nth_items(AA_PerfectScores)
MeanLearningCurve = []
for i,item in enumerate(TrialbyTrial_Pscores):
    MeanLearningCurve = MeanLearningCurve + [np.mean(item)]

fig,ax = plt.subplots(1, 1, figsize=(20, 10))
ax.set_ylim([0, 1])
ax.set_xlim([0, 7000])
for i in range(len(AA_PerfectScores)):
    ax.plot(convolve_movmean(AA_PerfectScores[i],window), color = 'teal',alpha = 0.2) 

patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])
ax.plot(convolve_movmean(MeanLearningCurve,window), color = 'teal')
   
ax.set_title('Perfect Scores Across Animals',fontsize = 20)
ax.set_xlabel('Trials',fontsize = 20)
ax.set_ylabel('P_score',fontsize = 20)

SaveFig('Perfect_scores_all_curves'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')
 

################### plot mean curve Perfect scores + stabdard deviation

    
# calculate standard deviation on pre-convolved p scores (working it out on raw data leads to huge over estimations of st-dev)
items = []
for i in range(len(AA_PerfectScores)):
    items = items + [convolve_movmean(AA_PerfectScores[i],window)]
stdLearningCurve= []
for item in conactinate_nth_items(items):
    stdLearningCurve = stdLearningCurve + [np.std(item)]
    
    
fig,ax = plt.subplots(1, 1, figsize=(20, 10))
ax.set_ylim([0, 1])
ax.set_xlim([0, 7000])
ax.plot(convolve_movmean(MeanLearningCurve,50), color = 'teal',alpha = 0.9) 
# ax.plot(convolve_movmean(upper,50), color = 'yellow',alpha = 0.9) 
# ax.plot(convolve_movmean(lower,50), color = 'red',alpha = 0.9) 

patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])

ax.set_xlabel('Trials',fontsize = 20)
ax.set_ylabel('P_score',fontsize = 20)

upper = np.array(MeanLearningCurve) + np.array(stdLearningCurve)
lower = np.array(MeanLearningCurve) - np.array(stdLearningCurve)
ax.fill_between(list(range(0,len(upper))),convolve_movmean(lower,window),convolve_movmean(upper,window),
    alpha=0.1, edgecolor='None', facecolor='teal',
    linewidth=1, linestyle='dashdot', antialiased=True)

ax.set_title('Mean Perfect Scores Across Animals [+ standard deviation]',fontsize = 20)

SaveFig('Perfect_scores_stdev'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')
 
    
################################################################################################################
##### plot transition scores:
################################################################################################################

transitions = [[],[],[],[]]
for item in AA_Transitionscore_trials:
    for index,transition in enumerate(item):
        transitions[index] = transitions[index] + [transition] 

MeanLearningCurve = [[],[],[],[]]
for ind in range(4):
    TrialbyTrial_Tscores = conactinate_nth_items(transitions[ind])
    for i,item in enumerate(TrialbyTrial_Tscores):
        MeanLearningCurve[ind] = MeanLearningCurve[ind] + [np.mean(item)]

fig,axs = plt.subplots(2, 2, figsize=(20, 20))
for ind, ax in enumerate(fig.axes):
    ax.plot(convolve_movmean(MeanLearningCurve[ind],100),color = 'teal')
    for item in (transitions[ind]):
        ax.plot(convolve_movmean(item,100),color = 'teal', alpha = 0.2)
    ax.set_ylim([0, 1])
    ax.set_xlim([0, 7000])
    
    ax.set_title('Transition ' + str(ind+1),fontsize = 20)
    
    ax.set_xlabel('Trials',fontsize = 20)
    ax.set_ylabel('T_score',fontsize = 20)
    
patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])

SaveFig('Transition_scores_allcurves'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')
 
    

#########transition scores with stdev ##########

# calculate standard deviation on pre-convolved p scores (working it out on raw data leads to huge over estimations of st-dev)

    
fig,axs = plt.subplots(2, 2, figsize=(20, 20))
for ind, ax in enumerate(fig.axes):
    ax.plot(convolve_movmean(MeanLearningCurve[ind],100), color = 'teal')
    
    ax.set_ylim([0, 1])
    ax.set_xlim([0, 7000])
    
    ax.set_xlabel('Trials',fontsize = 20)
    ax.set_ylabel('T_score',fontsize = 20)

    ### standard deviation:
    items = []
    for item in transitions[ind]:
        items = items + [convolve_movmean(item,window)]
    stdLearningCurve= []
    for item in conactinate_nth_items(items):
        stdLearningCurve = stdLearningCurve + [np.std(item)]
        
    ax.set_title('Transition ' + str(ind+1),fontsize = 20)

    upper = np.array(MeanLearningCurve[ind]) + np.array(stdLearningCurve)
    lower = np.array(MeanLearningCurve[ind]) - np.array(stdLearningCurve)
    ax.fill_between(list(range(0,len(upper))),convolve_movmean(lower,window),convolve_movmean(upper,window),
    alpha=0.1, edgecolor='None', facecolor='teal',
    linewidth=1, linestyle='dashdot', antialiased=True)
    
patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])
     
SaveFig('Transition_scores_+standardev'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')


################################################################################################################
##### plot number of trials per session (non cumulative and cumulative) across animals 
################################################################################################################

mean_trials_per_sesh = []
std_trials_per_sesh = []
for item in conactinate_nth_items(AA_TrialsPerSessions):
    mean_trials_per_sesh = mean_trials_per_sesh + [(np.mean(item))]
    std_trials_per_sesh = std_trials_per_sesh + [np.std(item)]
    
upper = np.array(mean_trials_per_sesh) + np.array(std_trials_per_sesh)
lower = np.array(mean_trials_per_sesh) - np.array(std_trials_per_sesh)



fig,[ax1,ax2] = plt.subplots(2, 1, figsize=(20, 20))

ax1.plot(mean_trials_per_sesh, color = 'teal',alpha = 0.9) 

patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])

ax1.set_xlabel('Session',fontsize = 20)
ax1.set_ylabel('Trials',fontsize = 20)

ax1.set_title('Trials per session [+ standard deviation]',fontsize = 20)

ax1.fill_between(list(range(0,len(upper))),(lower),(upper),
    alpha=0.1, edgecolor='None', facecolor='teal',
    linewidth=1, linestyle='dashdot', antialiased=True)



cum_mean_trials_per_sesh = []
cum_std_trials_per_sesh = []

all_items= []
for item in AA_TrialsPerSessions:
    all_items = all_items + [np.cumsum(item)]
for item in conactinate_nth_items(all_items):
    cum_mean_trials_per_sesh = cum_mean_trials_per_sesh + [(np.mean(item))]
    cum_std_trials_per_sesh = cum_std_trials_per_sesh + [np.std(item)]
    
upper = np.array(cum_mean_trials_per_sesh) + np.array(cum_std_trials_per_sesh)
lower = np.array(cum_mean_trials_per_sesh) - np.array(cum_std_trials_per_sesh)

ax2.plot(cum_mean_trials_per_sesh, color = 'teal',alpha = 0.9) 

patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
ax1.legend(handles=[patch1])
ax2.legend(handles=[patch1])


# for i in range(len(AA_TrialsPerSessions)):
#     ax2.plot(np.cumsum(AA_TrialsPerSessions[i]))

ax2.set_xlabel('Session',fontsize = 20)
ax2.set_ylabel('Trials',fontsize = 20)

ax2.fill_between(list(range(0,len(upper))),(lower),(upper),
    alpha=0.1, edgecolor='None', facecolor='teal',
    linewidth=1, linestyle='dashdot', antialiased=True)

ax2.set_title('Cumulative Trials per session [+ standard deviation]',fontsize = 20)
    
SaveFig('Trials_Per_session_'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')
    

################################################################################################################
#### Transition times  in in
################################################################################################################

transitions = [[],[],[],[]]
for item in AA_TransitionTimes_inin:
    for index,transition in enumerate(item):
        if index < 4:
            transitions[index] = transitions[index] + [transition] 

MeanLearningCurve = [[],[],[],[]]
for ind in range(4):
    TrialbyTrial_Tscores = conactinate_nth_items(transitions[ind])
    for i,item in enumerate(TrialbyTrial_Tscores):
        MeanLearningCurve[ind] = MeanLearningCurve[ind] + [np.mean(item)]

fig,axs = plt.subplots(4, 1, figsize=(10, 20))
for ind, ax in enumerate(fig.axes):
    ax.plot(convolve_movmean(MeanLearningCurve[ind],100),color = 'teal')
    for item in (transitions[ind]):
        ax.plot(convolve_movmean(item,100),color = 'teal', alpha = 0.2)
#     ax.set_ylim([0, 2])
    ax.set_xlim([0, 7000])
    
    ax.set_title('Transition ' + str(ind+1),fontsize = 20,loc = 'left')
    
    ax.set_xlabel('Trials',fontsize = 20)
    ax.set_ylabel('Time',fontsize = 20)
    
patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])

SaveFig('Transition_times_inin_allcurves'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')

#########transition scores with stdev ##########

# calculate standard deviation on pre-convolved p scores (working it out on raw data leads to huge over estimations of st-dev)
    
fig,axs = plt.subplots(4, 1, figsize=(10, 20))
for ind, ax in enumerate(fig.axes):
    ax.plot(convolve_movmean(MeanLearningCurve[ind],100), color = 'teal')
    
#     ax.set_ylim([0, 2])
    ax.set_xlim([0, 7000])
    
    ax.set_xlabel('Trials',fontsize = 20)
    ax.set_ylabel('Time',fontsize = 20)

    ### standard deviation:
    items = []
    for item in transitions[ind]:
        items = items + [convolve_movmean(item,window)]
    stdLearningCurve= []
    for item in conactinate_nth_items(items):
        stdLearningCurve = stdLearningCurve + [np.std(item)]
        
    ax.set_title('Transition ' + str(ind+1),fontsize = 20, loc = 'left')

    upper = np.array(MeanLearningCurve[ind]) + np.array(stdLearningCurve)
    lower = np.array(MeanLearningCurve[ind]) - np.array(stdLearningCurve)
    ax.fill_between(list(range(0,len(upper))),convolve_movmean(lower,window),convolve_movmean(upper,window),
    alpha=0.1, edgecolor='None', facecolor='teal',
    linewidth=1, linestyle='dashdot', antialiased=True)
    
patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])
     
SaveFig('Transition_times_inin +standardev'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')

################################################################################################################
#### Transition times  out in
################################################################################################################

transitions = [[],[],[],[]]
for item in AA_TransitionTimes_outin:
    for index,transition in enumerate(item):
        if index < 4:
            transitions[index] = transitions[index] + [transition] 

MeanLearningCurve = [[],[],[],[]]
for ind in range(4):
    TrialbyTrial_Tscores = conactinate_nth_items(transitions[ind])
    for i,item in enumerate(TrialbyTrial_Tscores):
        MeanLearningCurve[ind] = MeanLearningCurve[ind] + [np.mean(item)]

fig,axs = plt.subplots(4, 1, figsize=(10, 20))
for ind, ax in enumerate(fig.axes):
    ax.plot(convolve_movmean(MeanLearningCurve[ind],100),color = 'teal')
    for item in (transitions[ind]):
        ax.plot(convolve_movmean(item,100),color = 'teal', alpha = 0.2)
#     ax.set_ylim([0, 2])
    ax.set_xlim([0, 7000])
    
    ax.set_title('Transition ' + str(ind+1),fontsize = 20,loc = 'left')
    
    ax.set_xlabel('Trials',fontsize = 20)
    ax.set_ylabel('Time',fontsize = 20)
    
patch1 = mpatches.Patch(color='teal', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])

SaveFig('Transition_times_outin_allcurves'+Cohort_name+'_'+Animal_ID[0]+'_to_'+Animal_ID[-1]+'.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')


################################################################################################################
## Training levels  
window = 100
xlim = 4000
################################################################################################################

BL_MeanLearningCurve = mean_learning_curve(AA_TrainingLevels,Animal_ID)

standard_dev = []
for item in conactinate_nth_items(AA_TrainingLevels):
    standard_dev = standard_dev+ [np.std(item)]

fig,ax = plt.subplots(1, 1, figsize=(10, 10))
# ax.set_ylim([0, 1])
# ax.set_xlim([0, 7000])

ax.set_ylim([0, 50])
    
ax.set_xlim([0, xlim])
ax.plot(convolve_movmean(BL_MeanLearningCurve,window), '--',color = 'black',linewidth = 2)

upper = np.array(BL_MeanLearningCurve) + np.array(standard_dev)
lower = np.array(BL_MeanLearningCurve) - np.array(standard_dev)

ax.fill_between(list(range(0,len(upper))),convolve_movmean(lower,window),convolve_movmean(upper,window),
alpha=0.1, edgecolor='None', facecolor='grey',
linewidth=1, linestyle='dashdot', antialiased=True)

   
ax.set_title('Training levels across animals',fontsize = 20)
ax.set_xlabel('Trials',fontsize = 20)

ax.set_ylabel('Training level',fontsize = 20)

ax.tick_params(axis="y", labelsize=15)
ax.tick_params(axis="x", labelsize=15)

# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

ax.tick_params(axis="y", labelsize=15)
ax.tick_params(axis="x", labelsize=15)
    

SaveFig('Tlevels.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')


transitions = [[],[],[],[]]
for item in AA_TransitionTimes_inin:
    for index,transition in enumerate(item):
        if index < 4:
            transitions[index] = transitions[index] + [transition] 

MeanLearningCurve = [[],[],[],[]]
for ind in range(4):
    TrialbyTrial_Tscores = conactinate_nth_items(transitions[ind])
    for i,item in enumerate(TrialbyTrial_Tscores):
        MeanLearningCurve[ind] = MeanLearningCurve[ind] + [np.mean(item)]

#########transition times with stdev ##########

# calculate standard deviation on pre-convolved p scores (working it out on raw data leads to huge over estimations of st-dev)
colors = ['black','red','green','blue']

fig,ax = plt.subplots(1, 1, figsize=(10, 10))
for ind, item in enumerate(MeanLearningCurve):
    ax.plot(convolve_movmean(MeanLearningCurve[ind],100), color = colors[ind])
    
#     ax.set_ylim([0, 2])
    ax.set_xlim([0, 4000])
    
    ax.set_xlabel('Trials',fontsize = 20)
    ax.set_ylabel('Time',fontsize = 20)

    ### standard deviation:
    items = []
    for item in transitions[ind]:
        items = items + [convolve_movmean(item,window)]
    stdLearningCurve= []
    for item in conactinate_nth_items(items):
        stdLearningCurve = stdLearningCurve + [np.std(item)]

    upper = np.array(MeanLearningCurve[ind]) + np.array(stdLearningCurve)
    lower = np.array(MeanLearningCurve[ind]) - np.array(stdLearningCurve)
            

    ax.fill_between(list(range(0,len(upper))),convolve_movmean(lower,window),convolve_movmean(upper,window),
    alpha=0.1, edgecolor='None', facecolor=colors[ind],
    linewidth=1, linestyle='dashdot', antialiased=True)
    
# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

ax.tick_params(axis="y", labelsize=15)
ax.tick_params(axis="x", labelsize=15)
    

SaveFig('Times.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')


TrialbyTrial_Pscores = conactinate_nth_items(AA_PerfectScores)
MeanLearningCurve = []
for i,item in enumerate(TrialbyTrial_Pscores):
    MeanLearningCurve = MeanLearningCurve + [np.mean(item)]

# calculate standard deviation on pre-convolved p scores (working it out on raw data leads to huge over estimations of st-dev)
items = []
for i in range(len(AA_PerfectScores)):
    items = items + [convolve_movmean(AA_PerfectScores[i],window)]
stdLearningCurve= []
for item in conactinate_nth_items(items):
    stdLearningCurve = stdLearningCurve + [np.std(item)]
    
    
fig,ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_ylim([0, 1])
ax.set_xlim([0, 4000])
ax.plot(convolve_movmean(MeanLearningCurve,50),color = 'black',alpha = 0.9) 
# ax.plot(convolve_movmean(upper,50), color = 'yellow',alpha = 0.9) 
# ax.plot(convolve_movmean(lower,50), color = 'red',alpha = 0.9) 

patch1 = mpatches.Patch(color='black', label=('n = ' + str(len(Animal_ID))))
plt.legend(handles=[patch1])

ax.set_xlabel('Trials',fontsize = 20)
ax.set_ylabel('P_score',fontsize = 20)

upper = np.array(MeanLearningCurve) + np.array(stdLearningCurve)
lower = np.array(MeanLearningCurve) - np.array(stdLearningCurve)

ax.fill_between(list(range(0,len(upper))),convolve_movmean(lower,window),convolve_movmean(upper,window),
    alpha=0.1, edgecolor='None', facecolor='grey',
    linewidth=1, linestyle='dashdot', antialiased=True)

# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

ax.tick_params(axis="y", labelsize=15)
ax.tick_params(axis="x", labelsize=15)



SaveFig('Performance.png',OutputPath + '\\AcrossAnimals\\' + Cohort_name + '\\')


print('Done!')

Plotting...
Done!
